In [28]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

from keras import layers
from keras import regularizers

import re

In [29]:
from keras import backend as K
K.clear_session()

In [30]:
physical_devices = tf.config.list_physical_devices()
CLASSES = 48
IMAGE_SIZE = 24

print(physical_devices)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [31]:
tf.config.set_visible_devices(physical_devices[1], 'GPU')

## Building Dataset

### Attributes description

- a1 - face containing flag: (1-with face, 0-without face),
- a2 - image number in current class (person) beginning from 0,
- a3 - class (person) number beginning from 0,
- a4 - sex (0 - woman, 1 - man)
- a5 - race (0- white, 1 - negro, 2 - indian, ...)
- a6 - age (0 - baby, 1 - young, 2 - middle-age, 3 - old) 
- a7 - binokulars (0 - without, 1 - transparent, 2 - dark)
- a8 - emotional expression (not state!) (0 - sad, 1 - neutral, 2 - happy)

In [32]:
N_TRAIN_EXAMPLES=int(len(X) * 0.8)
N_TEST_EXAMPLES=len(X) - N_TRAIN_EXAMPLES

In [33]:
def read_file(filename):
  
  with open(filename, 'r') as file:
    lines = file.readlines()
    
    # Remove newLines
    for i, line in enumerate(lines):
      lines[i] = line.replace('\n', '')
    
    # We assume these are integers
    EXAMPLES_NR = int(lines[0])
    PIXELS_NR = int(lines[1])
    
    inputs = np.zeros(EXAMPLES_NR, dtype=dict)
    labels = np.zeros(EXAMPLES_NR, dtype=int)
    
    examples_raw = lines[2:EXAMPLES_NR]
    
    for i, example_raw in enumerate(examples_raw):
      # Split by spaces (treats multiple as one)
      tokens = re.split('\s+', example_raw)
      
      pixel_values = np.array(tokens[0:PIXELS_NR])
      attributes = tokens[PIXELS_NR:]       
        
      for j, el in enumerate(pixel_values):
        pixel_values[j] = float(el)
        
      pixel_values = pixel_values.reshape([IMAGE_SIZE, IMAGE_SIZE])
      
      # Add everything to the dataset dictionary
      
      input_dict = {
        "pixel_values": pixel_values,
        "face": attributes[0],
        "image_number": attributes[1],
        "sex": attributes[3],
        "race": attributes[4],
        "age": attributes[5],
        "binoculars": attributes[6],
        "emotion": attributes[7],
        # "class": attributes[2],
      }
      inputs[i] = input_dict
      labels[i] = int(attributes[2])
      
  return inputs, labels

In [34]:
# classes 0-15
X_0, y_0 = read_file('./data/x24x24.txt')
# classes 16-31
X_1, y_1 = read_file('./data/y24x24.txt')
# # classes 32-48
X_2, y_2 = read_file('./data/z24x24.txt')

# print(len(dataset_x), len(dataset_y), len(dataset_z))

In [35]:
from sklearn.model_selection import train_test_split

# Concatenate train and test images
X = np.concatenate((X_0, X_1, X_2))
y = np.concatenate((y_0, y_1, y_2))


X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=N_TRAIN_EXAMPLES, test_size=N_TEST_EXAMPLES, random_state=1)

In [36]:
print(len(X_test))

1367


## Building a model

AlexNet implementation in keras:

```py
  keras.Input(shape=(227, 227, 3)),
  layers.Conv2D(96, kernel_size=11, strides=4, padding='same', activation="relu"),
  layers.MaxPooling2D(pool_size=3, strides=2),
  layers.Conv2D(256, kernel_size=5, padding='same', activation="relu"),
  layers.MaxPooling2D(pool_size=3, strides=2),
  layers.Conv2D(384, kernel_size=3, padding='same', activation="relu"),
  layers.Conv2D(384, kernel_size=3, padding='same', activation="relu"),
  layers.Conv2D(256, kernel_size=3, padding='same', activation="relu"),
  layers.MaxPooling2D(pool_size=3, strides=2),
  layers.Dropout(0.5),
  layers.Dense(4096, activation="relu"),
  layers.Dropout(0.5),
  layers.Dense(4096, activation="relu"),
  layers.Dense(CLASSES, activation="softmax")
```

In [40]:
conv_regularizer = regularizers.l2(l=0.01)
dense_regularizer = regularizers.l2(l=0.01)

model = keras.Sequential(
  [
    keras.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 1)),
    layers.Conv2D(80, kernel_size=3, strides=1, padding='same', activation="relu", kernel_regularizer=conv_regularizer),
    layers.MaxPooling2D(pool_size=2, strides=1),
    layers.Conv2D(112, kernel_size=5, padding='same', activation="relu", kernel_regularizer=conv_regularizer),
    layers.MaxPooling2D(pool_size=3, strides=2),
    layers.Conv2D(160, kernel_size=3, padding='same', activation="relu", kernel_regularizer=conv_regularizer),
    layers.Conv2D(160, kernel_size=3, padding='same', activation="relu", kernel_regularizer=conv_regularizer),
    layers.Conv2D(112, kernel_size=3, padding='same', activation="relu", kernel_regularizer=conv_regularizer),
    layers.MaxPooling2D(pool_size=3, strides=2),
    layers.Flatten(),
    layers.Dropout(0.4),
    layers.Dense(768, activation="relu", kernel_regularizer=dense_regularizer),
    layers.Dropout(0.4),
    layers.Dense(768, activation="relu", kernel_regularizer=dense_regularizer),
    layers.Dense(CLASSES, activation="softmax"),
    
  ]
)

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 24, 24, 80)        800       
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 23, 23, 80)       0         
 2D)                                                             
                                                                 
 conv2d_16 (Conv2D)          (None, 23, 23, 112)       224112    
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 11, 11, 112)      0         
 g2D)                                                            
                                                                 
 conv2d_17 (Conv2D)          (None, 11, 11, 160)       161440    
                                                                 
 conv2d_18 (Conv2D)          (None, 11, 11, 160)      

## Training Preparation